In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable, vstack
import astropy.units as u
import sys
import pyneb as pn
from multiprocessing import Pool
import multiprocessing as mp
import math
from astropy.io import fits
from orcs.process import SpectralCube

from astropy.nddata import NDData, Cutout2D
from astropy.wcs import WCS

import astropy.units as u
from astropy.coordinates import SkyCoord

from reproject import reproject_interp
from regions import PixCoord

import pylab as pl

from scipy.optimize import curve_fit
from scipy.integrate import quad, simpson

from orb.fit import fit_lines_in_spectrum
from orb.utils.spectrum import corr2theta, amp_ratio_from_flux_ratio
from orb.core import Lines
import gvar
import orb

import extinction
from extinction import apply, remove

### Import data

In [ ]:
galaxynum = 5
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}  #There is no SITELLE data for NGC 4254, NGC 2835 has the best data 
galaxy = galdic[galaxynum]
print(galaxy)

galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651}
galvel = galveldic[galaxy]
galebv = {'NGC4254': 0.0334 , 'NGC4535': 0.0168 , 'NGC3351': 0.0239, 'NGC2835': 0.0859, 'NGC0628': 0.0607}
ebv = galebv[galaxy]

infile = open("/home/habjan/jupfiles/data/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
musedata = hdul[hdul['gal_name'] == f'{galaxy}']

hdul = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra.fits')

lam = hdul[0].data
spec = hdul[1].data - hdul[2].data

infile = fits.open(f'/home/habjan/jupfiles/data/Nebulae_Catalogue_DR2_native_with_OII.fits')
hdul = Table.read(infile)
fabian = hdul[hdul['gal_name'] == f'{galaxy}']

infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

# WCS info for SITELLE
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_cube.fits")
header = hdul[0].header
wcs = WCS(header,naxis=2)

# MUSE WCS info 
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_IMAGE_FOV_Johnson_B_WCS_Pall_mad.fits")
muse_data = NDData(data=hdul['DATA'].data, mask=np.isnan(hdul['DATA'].data), meta=hdul['DATA'].header, wcs=WCS(hdul['DATA'].header))    
muse_data.data[muse_data.data==0] = np.nan

# Halpha masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_MAPS.fits")
Halpha = NDData(data=hdul['HA6562_FLUX'].data, mask=np.isnan(hdul['HA6562_FLUX'].data), meta=hdul['HA6562_FLUX'].header, wcs=WCS(hdul['HA6562_FLUX'].header))
Halpha.data[muse_data.data==0] = np.nan

# Import the HII region spatial masks
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_nebulae_mask_V2.fits")
nebulae_mask = NDData(data = hdul[0].data.astype(float), mask=Halpha.mask, meta=hdul[0].header, wcs=WCS(hdul[0].header)) 
nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# SITELLE deepframe 
hdul = fits.open(f"/home/habjan/jupfiles/data/{galaxy}_deepframe.fits")
deepframe = NDData(data=hdul[0].data, meta=hdul[0].header, wcs=wcs)

### Make a list of moderately bright HII regions

In [ ]:
midlist = []
for i in range(len(musedata)):
    if musedata[i]['OII7319_FLUX'] > 8*musedata[i]['OII7319_FLUX_ERR'] and musedata[i]['OII7319_FLUX'] < 10*musedata[i]['OII7319_FLUX_ERR']:
        midlist.append(i)
midlist = np.array(midlist)
midlist

### Plot the fits of [OII]3729 for some of the higher S/N HII regions

In [ ]:
broadening_gvar = gvar.gvar(1, 20)

S2 = pn.Atom('S', 2)
O2 = pn.Atom('O', 2)
temp = 10**4
fitshape = 'sinc'
OII3726 = Lines().get_line_cm1('[OII]3726')
OII3729 = Lines().get_line_cm1('[OII]3729')
for i in midlist[0:5]:
        
      inspectrum = spec[i]
      inspectrum[np.isnan(inspectrum)] = 0

      wave3729 = 3728.815
      red3729 = 1/((wave3729*(musedata[i]['HA6562_VEL']+galvel)/(299792) + wave3729) * 10**-8)

      n1 = [np.where(lam > cube.params.filter_range[0])[0][0],np.where(lam > red3729 - 175)[0][0]]
      n2 = [np.where(lam > red3729 + 175)[0][0], np.where(lam > cube.params.filter_range[1])[0][0]]
      noise = (np.mean(inspectrum[n1[0]:n1[1]]) + np.mean(inspectrum[n2[0]:n2[1]])) / 2
      noisestd = (np.std(inspectrum[n1[0]:n1[1]]) + np.std(inspectrum[n2[0]:n2[1]])) / 2
      inspectrum = (inspectrum) - (noise)

      invel = musedata[i]['HA6562_VEL']+galvel
      velocity_gvar = gvar.gvar(invel, 3)
        
      fit = fit_lines_in_spectrum(inspectrum, [OII3729], 
                      step=cube.params.step, order=cube.params.order, nm_laser=cube.params.nm_laser, theta=corr2theta(cube.params.axis_corr), zpd_index=cube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def=['1'],
                      pos_cov=[velocity_gvar],
                      amp_def=['1'], 
                      amp_guess=[1],
                      )
      
      oiifit = fit['fitted_models']['Cm1LinesModel'][0]
      oii_flux = fit['flux'][0]
      oii_snr = fit['snr'][0]
      my_oii_snr = fit['lines_params'][0][1] / noisestd

      red3729 = fit['lines_params'][0][2]

      flux_rang = [np.where(red3729 - 125 < lam)[0][0],np.where(red3729 + 125 < lam)[0][0]]
      my_flux = simpson(oiifit[flux_rang[0]:flux_rang[1]])

      fig = plt.figure()
      fig.set_size_inches(10, 4)

      plt.title(f'Region {i} in {galaxy}')
      plt.plot(lam, inspectrum, color='blue', label='Noise subtracted spectrum')

      plt.axvline(red3729, color='red', linestyle='--', label='Redshifted [OII]3729')
      plt.plot(lam, oiifit, color='red', label='[OII] fit')

      plt.fill_between(lam[n1],-10**6, 10**10, color='purple', alpha = 0.15, label='Noise')
      plt.fill_between(lam[n2],-10**6, 10**10, color='purple', alpha = 0.15)
      plt.fill_between(lam[flux_rang],-10**6, 10**10, color='green', alpha = 0.15, label='Integration Range')

      plt.xlim(lam[n1[0]+5], lam[n2[1]-5])
      plt.ylim(0-np.max(inspectrum)*0.3, np.max(inspectrum)*1.7)

      plt.figtext(0.5, -0.175, f"[OII] Flux: {round(oii_flux, 3)}, SNR: {round(oii_snr,3)} \n My calculated SNR: {round(my_oii_snr,3)}\n My calculated Flux: {round(my_flux,3)}", ha="center", fontsize=10, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

      plt.xlabel(r'Wavenumber [cm$^{-1}$]')
      plt.ylabel(r'Flux [$\frac{1e-20 \cdot erg}{cm^{2} \cdot s \cdot cm^{-1}}$]') #\frac{1e-20 \cdot 
      plt.legend()

### The code below is a function to fit the [OII]3729 feature

In [ ]:
def fit3729(inwave, influx, snval, phdata, galvel): #, mcit):
    
    def gaussian(x, C, a, sig):    #a:amplitude, x0:average wavelength, delwave:influxtral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * sig ** 2)) + C
    
    def gaussian_noC(x, a, sig):
        return a * np.exp((-(x-w0) ** 2)/ (2 * sig ** 2))

    wavelength = 3729
    dopv = 126.47 - 0.00978*wavelength
    delwave = (wavelength * dopv * 10**13) / (3 * 10**8 * 10**10)

    outflux = np.zeros(len(phdata))

    for i in range(len(influx)):
        w0 = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = w0 - 20*delwave
        highb = w0 + 20*delwave
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(influx[i][np.where(inwave > w0 + 20)[0][0]:np.where(inwave > w0 + 100)[0][0]])
        p0list = np.array([fluxaves, 5000, delwave])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)

            flux = quad(gaussian_noC, w0 - 15 * delwave, w0 + 15 * delwave, args=(param[1], param[2]))[0]
            signal = param[1]
            noise1 = np.std(influx[i][np.where(inwave > w0 - 60)[0][0]:np.where(inwave > w0 - 10)[0][0]])
            noise2 = np.std(influx[i][np.where(inwave > w0 + 20)[0][0]:np.where(inwave > w0 + 100)[0][0]])
            noise = np.mean([noise1, noise2])

            if signal/noise > snval and flux > 0: 
                outflux[i] = flux - (flux/1.4040668586)
                #outfluxerr[i] = mcerr7319(inwave, influx[i], noise, w0, mcit)
            else: 
                outflux[i] = np.nan
                #outfluxerr[i] = np.nan

        except:
            outflux[i] = np.nan
            #outfluxerr[i] = np.nan
    
    return outflux#, outfluxerr

### The code below runs th [OII]3729 fitting function

In [ ]:
oiifits = fit3729(lam, spec, 3, musedata, galvel)
len(oiifits)

### Use the [OII]3729/3726 = 1.4 theoretical relationship to find each individual flux

In [ ]:
rat = 1.4040668586

flux = oiifits[1]

oii3729 = flux / (1 + 1/1.4)
oii3726 = flux / 2.4

oii3729, oii3726, oii3729/oii3726

### Match indices for HII regions that have different region IDs from different data releases

In [ ]:
rval = 4
fablist = []
mylist = []

for j in range(len(musedata)):
    for i in range(len(fabian)):
        if round(fabian[i]['cen_dec'], rval) == round(musedata[j]['cen_dec'], rval) and round(fabian[i]['cen_ra'], rval) == round(musedata[j]['cen_ra'], rval) and round(musedata[j]['deproj_dist']) == round(fabian[i]['deproj_dist']):
            fablist.append(i)
            mylist.append(j)

len(fablist), len(mylist)

### Plot the resultant flux values in comparison to Fabian's results

In [ ]:
#plt.scatter(oiifits[mylist], fabian[fablist]['OII3729_FLUX'], marker='.', s=50, c='k', alpha=0.5, label='[OII]3729')
plt.scatter(oiifits[mylist] / (1 + 1/1.4), fabian[fablist]['OII3729_FLUX'], marker='.', s=50, c='k', alpha=0.6, label='[OII]3729')
plt.scatter(oiifits[mylist] / (2.4), fabian[fablist]['OII3726_FLUX'], marker='.', s=50, c='red', alpha=0.4, label='[OII]3726')


plt.plot(np.linspace(1, 10**11, 10), np.linspace(1, 10**11, 10), c='k', label='1-to-1 line')

plt.title(f'[OII]3726,3729 Flux comparison in {galaxy}')
plt.ylabel('Fabians Fluxes')
plt.xlabel('Erics Fluxes')
lowb = 10**4
upb = 10**6
plt.xlim(lowb, upb)
plt.ylim(lowb, upb)
plt.xscale('log')
plt.yscale('log')
plt.legend()

# Use the ORBS package to fit both [OII] emission features

In [ ]:
np.where(musedata['OII7319_FLUX'] > 30*musedata['OII7319_FLUX_ERR'])

### Background emission

In [ ]:
regnum = 799
nebpix = (np.where(nebulae_mask.data == regnum)[0], np.where(nebulae_mask.data == regnum)[1])
edges = [np.max(nebpix[0]) - np.min(nebpix[0]), np.max(nebpix[1]) - np.min(nebpix[1])]
r = np.max(edges)/2
rmin = r * 3
rmax = np.sqrt(r**2 + rmin**2)
#rmax = r * 10
x = np.median(nebpix[0])
y = np.median(nebpix[1])

imin = x - rmax
imax = x + rmax + 1
jmin = y - rmax
jmax = y + rmax + 1
xlist = []
ylist = []
for i in np.arange(imin, imax):
    for j in np.arange(jmin, jmax):
        ij = np.array([i,j])
        dist = np.linalg.norm(ij - np.array((x,y)))
        i, j = int(i), int(j)
        distnum = 3
        if dist > rmin and dist <= rmax and np.all(np.isnan(nebulae_mask.data[i-distnum:i+distnum,j-distnum:j+distnum])):
            xlist.append(i)
            ylist.append(j)

inpix = (xlist, ylist)


testcoord = WCS(nebulae_mask.meta).pixel_to_world(inpix[1], inpix[0])
sitpix = np.transpose(cube.world2pix((testcoord.ra.degree, testcoord.dec.degree)))
sitpix = sitpix[0].astype(np.int64), sitpix[1].astype(np.int64)

back = cube.extract_integrated_spectrum(sitpix)
backspec = np.real(back[1])
backspec[np.isnan(backspec)] = 0
backspec = backspec

### The code below extracts a ratio for the [OII] fluxes

In [ ]:
S2 = pn.Atom('S', 2)
O2 = pn.Atom('O', 2)
temp = 10**4
S2ratio = musedata['SII6730_FLUX_CORR'] / musedata['SII6716_FLUX_CORR']
density = S2.getTemDen(int_ratio=S2ratio, tem=temp, wave1=6731, wave2=6717)
O2ratio = O2.getEmissivity(tem=temp, den=density, wave=3729) / O2.getEmissivity(tem=temp, den=density, wave=3726)

np.mean(O2ratio[np.where(~np.isnan(O2ratio))]) / 1.4

### The code below fits the spectrum

In [ ]:
region = 73
inspectrum = spec[region]
inspectrum[np.isnan(inspectrum)] = 0

wave3726 = 3726.032
red3726 = 1/((wave3726*(musedata[region]['HA6562_VEL']+galvel)/(299792) + wave3726) * 10**-8)
wave3729 = 3728.815
red3729 = 1/((wave3729*(musedata[region]['HA6562_VEL']+galvel)/(299792) + wave3729) * 10**-8)

wave = lam 
n1 = [np.where(wave > cube.params.filter_range[0])[0][0],np.where(wave > red3729 - 50)[0][0]]
n2 = [np.where(wave > red3729 + 50)[0][0], np.where(wave > cube.params.filter_range[1])[0][0]]
waven1 = wave[n1[0]:n1[1]]
waven2 = wave[n2[0]:n2[1]]
noise = (np.mean(inspectrum[n1[0]:n1[1]]) + np.mean(inspectrum[n2[0]:n2[1]])) / 2

OII3726 = Lines().get_line_cm1('[OII]3726')
OII3729 = Lines().get_line_cm1('[OII]3729')

inspec = (inspectrum) - (noise)
#inspec[inspec < 0] = 0


S2 = pn.Atom('S', 2)
O2 = pn.Atom('O', 2)
temp = 10**4
S2ratio = musedata[region]['SII6730_FLUX_CORR'] / musedata[region]['SII6716_FLUX_CORR']
density = S2.getTemDen(int_ratio=S2ratio, tem=temp, wave1=6731, wave2=6717)
O2ratio = O2.getEmissivity(tem=temp, den=density, wave=3729) / O2.getEmissivity(tem=temp, den=density, wave=3726)
#O2ratio = 1.4
amp_ratio = amp_ratio_from_flux_ratio(OII3729,OII3726,O2ratio)

invel = musedata[region]['HA6562_VEL']+galvel
velocity_gvar = gvar.gvar(invel, 20)
broadening_gvar = gvar.gvar(1, 20)

fitshape = 'sinc'

#inspec = (inspectrum * 10**-20) - backspec

fit = fit_lines_in_spectrum(inspec, [red3726, red3729], 
                      step=cube.params.step, 
                      order=cube.params.order, 
                      nm_laser=cube.params.nm_laser, 
                      theta=corr2theta(cube.params.axis_corr), 
                      zpd_index=cube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def='fixed', #['1', '1'],
                      #pos_cov=[velocity_gvar],
                      amp_def=['1', '1'], 
                      amp_guess=[1, amp_ratio],
                      )

fit['flux'][1]/ fit['flux'][0], O2ratio

### Compare S/N values

In [ ]:
snrnoise = np.std(inspectrum[n1[0]: n1[1]]) + np.std(inspectrum[n2[0]:n2[1]]) 
snrnoise = snrnoise / 2
OII3726_fit, OII3729_fit = fit['fitted_models']['Cm1LinesModel']
sig = np.max(OII3726_fit + OII3729_fit)
mycalculatednoise = sig / snrnoise

orcssnr = fit['snr'][0]

print(f'My SNR is {round(mycalculatednoise, 3)} while the SNR from the ORCS module is {round(orcssnr, 3)} for the [OII]3726 flux')

### Comapre velocities from MUSE-PHANGS catalog and fitted velocity

In [ ]:
fitvel = fit['velocity'][0]
nebvel = musedata[region]['HA6562_VEL']+galvel
print(f'The velicty from the fit is {round(fitvel, 1)} km/s and the velocity from the Nebular catalog is {round(nebvel,1)} km/s')

### The code below plots the 'Bayesian' [OII] fits

In [ ]:
OII3726_fit, OII3729_fit = fit['fitted_models']['Cm1LinesModel']
OII3726_flux, OII3729_flux = fit['flux']
OII3726red, OII3729red = fit['lines_params'][0][2], fit['lines_params'][1][2]

fig = plt.figure()
fig.set_size_inches(10, 4)

plt.plot(wave, OII3726_fit, c='red', label=f'[OII]3726 Fit; Flux {round(OII3726_flux, 2)}')
plt.plot(wave, OII3729_fit, c='blue', label=f'[OII]3729 Fit; Flux {round(OII3729_flux, 2)}')
plt.plot(wave, inspec, c='k', label='Raw Integrated Spectrum')
#plt.plot(wave, fit['fitted_vector'])

plt.plot(wave, OII3726_fit+OII3729_fit, c='green', label=f'[OII]3729 Fit + [OII3726] Fit')

plt.fill_between(waven1,-10**6, 10**10, color='purple', alpha = 0.15, label='Noise')
plt.fill_between(waven2,-10**6, 10**10, color='purple', alpha = 0.15)

plt.axvline(OII3726, c='purple', alpha=0.7, linestyle='--', label=f'3726 Angstrom')
plt.axvline(OII3729, c='orange', alpha=0.7, linestyle='--', label=f'3729 Angstrom')
plt.axvline(OII3726red, c='red', alpha=0.7, linestyle='--', label=f'Redshifted 3726 Angstrom: {round(1/(OII3726red*10**-8), 2)}')
plt.axvline(OII3729red, c='blue', alpha=0.7, linestyle='--', label=f'Redshifted 3729 Angstrom: {round(1/(OII3729red*10**-8), 2)}')

plt.xlim(wave[n1[0]-1], wave[n2[1]]+1)
plt.ylim(0-np.max(inspec)*0.3, np.max(inspec)*1.7)

shapedict = {'gaussian':'Gaussian', 'sinc':'Sinc', 'sincgauss':'Sinc + Gaussian', 'sinc2':r'$/sqrt{sinc^{2}}$'}
inshape = shapedict[fitshape]

plt.title(f'Region {region} from {galaxy} fitted with {inshape} with Velocity {round(fitvel, 1)}'r'[$\frac{km}{s}$]')
plt.xlabel(r'Wavenumber [cm$^{-1}$]')
plt.ylabel(r'Flux [$\frac{1e-20 \cdot erg}{cm^{2} \cdot s \cdot Angstrom}$]') #\frac{1e-20 \cdot 
plt.legend()

### Create a function to find the [OII] fluxes in all the regions

In [ ]:
def oiiorc(phdata, inspec, ingalvel, incube, inwave, inebv, snval):

    outflux3726, outflux3729 = np.zeros(len(phdata)), np.zeros(len(phdata))
    outflux3726err, outflux3729err = np.zeros(len(phdata)), np.zeros(len(phdata))
    broadening_gvar = gvar.gvar(1, 20)

    fitshape = 'sinc'
    for i in range(len(phdata)):

        OII3726 = Lines().get_line_cm1('[OII]3726')
        OII3729 = Lines().get_line_cm1('[OII]3729')
        
        inspectrum = inspec[i]

        wave3726 = 3726.032
        red3726 = 1/((wave3726*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wave3726) * 10**-8)
        wave3729 = 3728.815
        red3729 = 1/((wave3729*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wave3729) * 10**-8)

        noise1 = np.mean(inspectrum[np.where(inwave > red3729 - 180)[0][0]:np.where(inwave > red3729 - 50)[0][0]])
        noise2 = np.mean(inspectrum[np.where(inwave > red3729 + 40)[0][0]:np.where(inwave > red3729 + 180)[0][0]])
        noise = np.mean([noise1, noise2])
        inspectrum[np.isnan(inspectrum)] = 0
        inspectrum = (inspectrum * 10**-20) - (noise * 10**-20)

        invel = phdata[i]['HA6562_VEL']+ingalvel
        velocity_gvar = gvar.gvar(invel, 20)
        amp_ratio = amp_ratio_from_flux_ratio(OII3729,OII3726,1.4)

        try:
            fit = fit_lines_in_spectrum(inspectrum, [OII3726, OII3729], 
                      step=incube.params.step, order=incube.params.order, nm_laser=incube.params.nm_laser, theta=corr2theta(incube.params.axis_corr), zpd_index=incube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def=['1', '1'],
                      pos_cov=[velocity_gvar],
                      amp_def=['1', '1'], 
                      amp_guess=[1, amp_ratio],
                      )
            if fit['snr'][0] > snval and fit['snr'][1] > snval:
                outflux3726[i], outflux3729[i] = fit['flux'] * 10**20
                outflux3726err[i], outflux3729err[i] = fit['flux_err'] * 10**20
            else:
                outflux3726[i], outflux3729[i] = np.nan, np.nan
                outflux3726err[i], outflux3729err[i] = np.nan, np.nan
        except: 
            outflux3726[i], outflux3729[i] = np.nan, np.nan
            outflux3726err[i], outflux3729err[i] = np.nan, np.nan
    
    R_V = 3.1
    mwcorr26 = remove(extinction.odonnell94(np.array([Lines().get_line_nm('[OII]3726') * 10]), ebv*R_V, R_V), outflux3726)
    mwcorr29 = remove(extinction.odonnell94(np.array([Lines().get_line_nm('[OII]3729') * 10]), ebv*R_V, R_V), outflux3729)
    mwcorr26err = remove(extinction.odonnell94(np.array([Lines().get_line_nm('[OII]3726') * 10]), ebv*R_V, R_V), outflux3726err)
    mwcorr29err = remove(extinction.odonnell94(np.array([Lines().get_line_nm('[OII]3729') * 10]), ebv*R_V, R_V), outflux3729err)

    return mwcorr26, mwcorr29, mwcorr26err, mwcorr29err

### Run the function

In [ ]:
flux3726, flux3729, flux3726err, flux3729err = oiiorc(musedata, spec, galvel, cube, lam, ebv, 0)

### Create lists that match the HII regions from Fabian's data to my data

In [ ]:
rval = 0
fablist = []
mylist = []
count = 0

#round(musedata[j]['deproj_dist'], rval) == round(fabian[i]['deproj_dist'], rval) and\

for j in range(len(musedata)):
    for i in range(len(fabian)):
        if round(fabian[i]['cen_dec'], rval) == round(musedata[j]['cen_dec'], rval) and round(fabian[i]['cen_ra'], rval) == round(musedata[j]['cen_ra'], rval) and\
        round(musedata[j]['cen_x_geo'], rval) == round(fabian[i]['cen_x_geo'], rval) and\
            round(musedata[j]['cen_y_geo'], rval) == round(fabian[i]['cen_y_geo'], rval):
            count += 1
            fablist.append(i)
            mylist.append(j)
    if count > 1:
        print(count, i)
    count = 0

len(fablist), len(mylist)

In [ ]:
np.mean(flux3729[mylist]/fabian[fablist]['OII3729_FLUX']), np.mean(flux3726[mylist]/fabian[fablist]['OII3726_FLUX'])

### Compare Fabian and I's results

In [ ]:
plt.scatter(flux3729[mylist], fabian[fablist]['OII3729_FLUX'], marker='.', s=50, c='k', alpha=0.6, label='[OII]3729')
plt.scatter(flux3726[mylist], fabian[fablist]['OII3726_FLUX'], marker='.', s=50, c='red', alpha=0.4, label='[OII]3726')

#plt.errorbar(flux3729[mylist], fabian[fablist]['OII3729_FLUX'], yerr=fabian[fablist]['OII3729_FLUX_ERR'], c='k',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3729')
#plt.errorbar(flux3726[mylist], fabian[fablist]['OII3726_FLUX'], yerr=fabian[fablist]['OII3726_FLUX_ERR'], c='red',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3726')

plt.plot(np.linspace(1, 10**11, 10), np.linspace(1, 10**11, 10), c='k', label='1-to-1 line')

plt.title(f'[OII]3726,3729 Flux comparison in {galaxy}')
plt.ylabel('Fabians Fluxes')
plt.xlabel('Erics Fluxes')
lowb = 10**3
upb = 10**7
plt.xlim(lowb, upb)
plt.ylim(lowb, upb)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()

### Comparing [OIII]5007 from each table as a sanity check

In [ ]:
plt.scatter(musedata[mylist]['HB4861_FLUX'], fabian[fablist]['HB4861_FLUX'], marker='.', s=50, c='k', alpha=0.6, label=r'H$\beta$')
plt.scatter(musedata[mylist]['OIII5006_FLUX'], fabian[fablist]['OIII5006_FLUX'], marker='.', s=50, c='red', alpha=0.4, label='[OIII]5007')
plt.scatter(musedata[mylist]['OII7319_FLUX'], fabian[fablist]['OII7319_FLUX'], marker='.', s=50, c='green', alpha=0.4, label='[OII]7319')

#plt.errorbar(flux3729[mylist], fabian[fablist]['OII3729_FLUX'], yerr=fabian[fablist]['OII3729_FLUX_ERR'], c='k',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3729')
#plt.errorbar(flux3726[mylist], fabian[fablist]['OII3726_FLUX'], yerr=fabian[fablist]['OII3726_FLUX_ERR'], c='red',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3726')

plt.plot(np.linspace(1, 10**11, 10), np.linspace(1, 10**11, 10), c='k', label='1-to-1 line')

plt.title(f'Flux comparison in {galaxy}')
plt.ylabel('HIIregion_cat_DR2_native.fits')
plt.xlabel('Nebulae_catalogue_v3.fits')
lowb = 10**1
upb = 10**7.5
plt.xlim(lowb, upb)
plt.ylim(lowb, upb)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()

# Comparison with the same data that Fabian used to derive his [OII] fluxes

### Import data

In [ ]:
infile = open("/home/habjan/jupfiles/data/HIIregion_cat_DR2_native.fits",'rb')
hdul = Table.read(infile)
musedata1 = hdul[hdul['gal_name'] == f'{galaxy}']

hdul = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra_HIIreg.fits')
lam1 = hdul[0].data
spec1 = hdul[1].data

### Fit [OII] emission lines

In [ ]:
flux3726_1, flux3729_1, flux3726err_1, flux3729err_1 = oiiorc(musedata1, spec1, galvel, cube, lam1, ebv, 0)

### Plot these results against Fabian's fluxes

In [ ]:
plt.scatter(flux3729_1, fabian['OII3729_FLUX'], marker='.', s=50, c='k', alpha=0.6, label='[OII]3729')
plt.scatter(flux3726_1, fabian['OII3726_FLUX'], marker='.', s=50, c='red', alpha=0.4, label='[OII]3726')

#plt.errorbar(flux3729[mylist], fabian[fablist]['OII3729_FLUX'], yerr=fabian[fablist]['OII3729_FLUX_ERR'], c='k',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3729')
#plt.errorbar(flux3726[mylist], fabian[fablist]['OII3726_FLUX'], yerr=fabian[fablist]['OII3726_FLUX_ERR'], c='red',alpha=0.35, fmt=".", markersize=8, capsize=5, label='[OII]3726')

plt.plot(np.linspace(1, 10**11, 10), np.linspace(1, 10**11, 10), c='k', label='1-to-1 line')

plt.title(f'[OII]3726,3729 Flux comparison in {galaxy}')
plt.ylabel('Fabians Fluxes')
plt.xlabel('Erics Fluxes')
lowb = 10**2
upb = 10**7
plt.xlim(lowb, upb)
plt.ylim(lowb, upb)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()

In [ ]:
np.mean(flux3729_1/fabian['OII3729_FLUX']), np.mean(flux3726_1/fabian['OII3726_FLUX'])